In [87]:
import os
import json
import pandas as pd
import traceback 

In [88]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()

True

In [89]:
KEY = os.getenv('OPENAI_API_KEY')

In [90]:
llm = ChatOpenAI(openai_api_key=KEY,model='gpt-3.5-turbo',temperature=0.3)

In [91]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [92]:
RESPONSE_JSON = {
    "1": {
        "mcq":"multiple choice question",
        "options": {
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
    "2": {
        "mcq":"multiple choice question",
        "options": {
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
    "3": {
        "mcq":"multiple choice question",
        "options": {
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
    

}

In [93]:
TEMPLATE ="""
Text:{text}
You are a expert MCQ maker. Given the above text , it is your job to \
create a quize of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be confirming the next as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs.
### RESPONSE_JSON
{response_json}

"""

In [94]:
quize_generation_prompt = PromptTemplate(
    input_variables = ['text','number','subject','tone','response_json'],
    template=TEMPLATE
)

In [95]:
quize_chain = LLMChain(llm=llm,prompt=quize_generation_prompt,output_key='generated_quiz',verbose=True)

In [96]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{generated_quiz}

Check from an expert English Writer of the above quiz:
"""

In [97]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "generated_quiz"], template=TEMPLATE2)

In [98]:
review_chain = LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key="review",verbose=True)

In [111]:
generate_evaluate_chain=SequentialChain(chains=[quize_chain,review_chain],input_variables=['text','number','subject','tone','response_json'],output_variables=['generated_quiz','review'],verbose=True,)

In [112]:
file_path = r"D:\GenAIProjects\MCQ_Generator\data.txt"

In [113]:
with open(file_path,'r') as f:
    TEXT = f.read()

In [114]:
print(TEXT)

The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[13] Other researchers who have studied human cognitive systems contributed

In [115]:
# Serilizes a python dictionary into a JSON formated string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [116]:
NUMBER = 5
SUBJECT = 'Machine Learning'
TONE = 'simple'

In [117]:
with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
            "text":TEXT,
            "number":NUMBER,
            "subject":SUBJECT,
            "tone":TONE,
            "response_json":json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used 

In [118]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1714
Prompt Tokens:1340
Completion Tokens:374
Total Cost:0.002758


In [131]:
response

{'text': 'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]\n\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[13] Other researchers who have studied human cognitive system

In [148]:
quiz=response.get('generated_quiz')

In [149]:
quiz=json.loads(quiz)
quiz

{'1': {'mcq': "Who coined the term 'machine learning'?",
  'options': {'a': 'Arthur Samuel',
   'b': 'Donald Hebb',
   'c': 'Walter Pitts',
   'd': 'Warren McCulloch'},
  'correct': 'a'},
 '2': {'mcq': 'What was the earliest machine learning model introduced by Arthur Samuel?',
  'options': {'a': 'Chess playing program',
   'b': 'Checkers winning chance calculator',
   'c': 'Speech pattern analyzer',
   'd': 'Sonar signal analyzer'},
  'correct': 'b'},
 '3': {'mcq': 'Which book introduced a theoretical neural structure that influenced machine learning algorithms?',
  'options': {'a': 'The Organization of Behavior',
   'b': 'Learning Machines',
   'c': 'Computing Machinery and Intelligence',
   'd': 'Research into Machine Learning'},
  'correct': 'a'},
 '4': {'mcq': "What was the purpose of the experimental 'learning machine' called Cybertron developed by Raytheon Company?",
  'options': {'a': 'Analyze sonar signals',
   'b': 'Analyze electrocardiograms',
   'c': 'Analyze speech pattern

In [151]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})
quiz_table_data   

[{'MCQ': "Who coined the term 'machine learning'?",
  'Choices': 'a: Arthur Samuel | b: Donald Hebb | c: Walter Pitts | d: Warren McCulloch',
  'Correct': 'a'},
 {'MCQ': 'What was the earliest machine learning model introduced by Arthur Samuel?',
  'Choices': 'a: Chess playing program | b: Checkers winning chance calculator | c: Speech pattern analyzer | d: Sonar signal analyzer',
  'Correct': 'b'},
 {'MCQ': 'Which book introduced a theoretical neural structure that influenced machine learning algorithms?',
  'Choices': 'a: The Organization of Behavior | b: Learning Machines | c: Computing Machinery and Intelligence | d: Research into Machine Learning',
  'Correct': 'a'},
 {'MCQ': "What was the purpose of the experimental 'learning machine' called Cybertron developed by Raytheon Company?",
  'Choices': 'a: Analyze sonar signals | b: Analyze electrocardiograms | c: Analyze speech patterns | d: All of the above',
  'Correct': 'd'},
 {'MCQ': 'What is one of the objectives of modern-day ma

In [152]:
import pandas as pd

quiz = pd.DataFrame(quiz_table_data)
quiz.to_csv('MachineLearning.csv',index=False)